In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel,  BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset

c:\Users\shree\Desktop\docu3c\Bollywood\Bollywood\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [3]:
df=pd.read_csv(r'C:\Users\shree\Desktop\docu3c\Bollywood\Bollywood-Data-master\wikipedia-data\coref_plot.csv')

In [4]:
len(df)

34728

In [5]:
df

,Unnamed: 0,Movie Name,Coref Plot
0,0,Aan Milo Sajna,Widowed and ailing Savitri Choudhury lives a ...
1,1,Aan Milo Sajna,Savitri knows that Anil is only waiting for S...
2,2,Aan Milo Sajna,The Diwan convinces Anil to mend Anil ways g...
3,3,Aan Milo Sajna,Shortly thereafter Anil does appear to have m...
4,4,Aan Milo Sajna,Deepali moves in the mansion looks after Sav...
...,...,...,...
34723,34723,Jo Hum Chahein,Rohan discovers Amrita and Vikram were just u...
34724,34724,Jo Hum Chahein,Rohan apologizes to Rohan father for all Roha...
34725,34725,Jo Hum Chahein,Neha was n't ready for a child and was only h...
34726,34726,Jo Hum Chahein,Rohan is shattered Neha apologizes and asks ...


In [6]:
plot=''
movie=[]
all_plots=[]
for i in range(len(df)):
    if df.loc[i,'Movie Name'] in movie:
        plot+=df.loc[i,'Coref Plot'].strip()+' '
    else:
        if len(movie)!=0:
            movie.pop()
            all_plots.append(plot)
            plot=''
        movie.append(df.loc[i,'Movie Name'])
        if i==0:
            plot=df.loc[i,'Coref Plot'].strip()


In [7]:
plot

"Rohan has set Rohan eyes on being a stockbroker in Mumbai because that 's the quickest way to make big money Rohan goes to Mumbai with Rohan best friend Abhay  a fellow MBA graduate  and on first night out Rohan meets Neha Kapoor and the two have a war of words  for the first time in Rohan life Rohan advances have been rejected by a girl Rohan  Abhay and 28 new stockbrokers start work at Bombay Bulls but after three months only the 10 best will survive and get permanent placements Rohan pitches clients but does n't climb the charts  Rohan remains near the bottom  so Rohan uses the help of Vikram Khurrana  the firm 's best stockbroker  to climb the charts faster Vikram introduces Rohan to Amrita Singhania  a wealthy society lady who is instantly taken by Rohan 's boyish charm  the same night Rohan also bumps into Neha and this time Rohan does n't budge until Neha agrees to go on a date with Rohan The next day  Amrita invites Rohan to a beach party where Amrita tries to seduce Rohan wit

In [8]:
all_plots

["Widowed and ailing Savitri Choudhury lives a wealthy lifestyle along with Savitri son  Anil  in a palatial mansion in IndiaSavitri knows that Anil is only waiting for Savitri to die so that Anil can inherit the wealth and refuses to give Anil any money The Diwan convinces Anil to mend Anil ways  get married  and patch-up with Anil mom Shortly thereafter Anil does appear to have mended Anil ways  gets involved in charity  and even introduces a young woman named Deepali to Anil mom Deepali moves in the mansion  looks after Savitri so much so that Savitri decides to make Savitri the sole beneficiary of Savitri estate What Savitri does not know is that Anil has hired Deepali to act as Anil fiancee  and that Deepali herself is not who Savitri claims to be  and is actually in love with a local horse-riding peasant  Ajit  whose father was convicted of killing Savitri 's husband  and soon Ajit Anil will be arrested by the Police for having an affair and then killing a woman named Sita ",
 "W

In [30]:
with open('dataset.txt','r+'):
    f=open('dataset.txt','r+')
    for i in range(len(all_plots)):
        f.write(all_plots[i])
        f.write('\n\n')
    f.close()

In [9]:
def get_input_ids_and_attention_mask(input_sequence):
    input_ids = torch.tensor([tokenizer.cls_token_id] + [tokenizer.convert_tokens_to_ids(token) for token in input_sequence] + [tokenizer.sep_token_id])
    attention_mask = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))
    return input_ids, attention_mask

In [10]:
def get_input_ids_and_attention_mask(input_sequence):
    input_ids = torch.tensor([tokenizer.cls_token_id] + [tokenizer.convert_tokens_to_ids(token) for token in input_sequence] + [tokenizer.sep_token_id])
    attention_mask = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))
    return input_ids, attention_mask

In [11]:
# class CustomDataset(Dataset):
#     def __init__(self, input_sequences):
#         self.input_sequences = input_sequences

#     def __len__(self):
#         return len(self.input_sequences)

#     def __getitem__(self, index):
#         input_sequence = self.input_sequences[index]
#         input_ids, attention_mask = get_input_ids_and_attention_mask(input_sequence)
#         return input_ids, attention_mask

In [12]:
plot1=all_plots[0]
plot1

"Widowed and ailing Savitri Choudhury lives a wealthy lifestyle along with Savitri son  Anil  in a palatial mansion in IndiaSavitri knows that Anil is only waiting for Savitri to die so that Anil can inherit the wealth and refuses to give Anil any money The Diwan convinces Anil to mend Anil ways  get married  and patch-up with Anil mom Shortly thereafter Anil does appear to have mended Anil ways  gets involved in charity  and even introduces a young woman named Deepali to Anil mom Deepali moves in the mansion  looks after Savitri so much so that Savitri decides to make Savitri the sole beneficiary of Savitri estate What Savitri does not know is that Anil has hired Deepali to act as Anil fiancee  and that Deepali herself is not who Savitri claims to be  and is actually in love with a local horse-riding peasant  Ajit  whose father was convicted of killing Savitri 's husband  and soon Ajit Anil will be arrested by the Police for having an affair and then killing a woman named Sita "

In [13]:
len(all_plots)

1877

In [15]:
with open('dataset.txt', 'r') as f:
    text_data = f.read()

In [16]:
dataset=text_data
dataset_size=len(dataset)

In [21]:
# optimizer = AdamW(model.parameters(), lr=1e-5)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=100)

c:\Users\shree\Desktop\docu3c\Bollywood\Bollywood\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# input_ids_tensor = torch.tensor([input_ids for input_ids, attention_mask in zip(input_ids_tensor, attention_mask_tensor)]).unsqueeze(1)
# attention_mask_tensor = torch.tensor([attention_mask for input_ids, attention_mask in zip(input_ids_tensor, attention_mask_tensor)]).unsqueeze(1)


# for epoch in range(5):
#     for input_ids, attention_mask in zip(input_ids_tensor, attention_mask_tensor):
#         outputs = model(input_ids, attention_mask=attention_mask)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
#         optimizer.zero_grad()

In [26]:
# model.save_pretrained('fine-tuned-bert')

In [20]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# input_texts = ["This is a movie review.", "This is another movie review."]
# input_ids = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")
# attention_mask = input_ids["attention_mask"]

# for epoch in range(5):
#     outputs = model(**input_ids, attention_mask=attention_mask)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

In [1]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# input_texts = ["This is a movie review.", "This is another movie review."]
# input_ids = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")
# attention_mask = input_ids["attention_mask"]

# for epoch in range(5):
#     outputs = model(**input_ids)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

In [2]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# input_texts = ["This is a movie review.", "This is another movie review."]
# input_ids = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")
# attention_mask = input_ids["attention_mask"]

# for epoch in range(5):
#     outputs = model(**input_ids)
#     logits = outputs.logits
#     labels = torch.tensor([1, 0]).unsqueeze(0)  # Assuming binary classification
#     loss_function = torch.nn.CrossEntropyLoss()
#     loss = loss_function(logits, labels)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

In [3]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch


# device=torch.device("cuda")
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# input_texts = ["This is a movie review.", "This is another movie review."]
# input_ids = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")
# attention_mask = input_ids["attention_mask"]

# for epoch in range(5):
#     outputs = model(**input_ids)
#     logits = outputs.logits
#     labels = torch.tensor([[1, 0], [0, 1]]).to(device)
#     loss_function = torch.nn.CrossEntropyLoss()
#     loss = loss_function(logits, labels)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_and_tokenize(movie_plots):
    lemmatizer = WordNetLemmatizer()

    stop_words = set(stopwords.words('english'))

    preprocessed_movie_plots = []
    for movie_plot in movie_plots:
        movie_plot = movie_plot.lower()
        tokens = nltk.word_tokenize(movie_plot)
        preprocessed_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
        preprocessed_movie_plot = ' '.join(preprocessed_tokens)
        preprocessed_movie_plots.append(preprocessed_movie_plot)

    return preprocessed_movie_plots

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shree\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shree\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
all_plots_processed=preprocess_and_tokenize(all_plots)

In [15]:
from transformers import BertTokenizer, BertModel
import torch

movie_plots = all_plots[:100] 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


inputs = tokenizer(movie_plots, return_tensors='pt', padding=True, truncation=True)


with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state





In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(embeddings.numpy().reshape(-1, embeddings.shape[-1]))

In [17]:
similarity_matrix

array([[ 1.0000001 ,  0.44230175,  0.20416589, ...,  0.22504821,
         0.3202901 ,  0.09010608],
       [ 0.44230175,  0.9999999 ,  0.24858047, ...,  0.04778552,
         0.24955294, -0.0461828 ],
       [ 0.20416589,  0.24858047,  0.9999999 , ...,  0.3672452 ,
         0.2619114 ,  0.14695796],
       ...,
       [ 0.22504821,  0.04778552,  0.3672452 , ...,  0.9999999 ,
         0.24551505,  0.2005297 ],
       [ 0.3202901 ,  0.24955294,  0.2619114 , ...,  0.24551505,
         0.9999999 ,  0.13797034],
       [ 0.09010608, -0.0461828 ,  0.14695796, ...,  0.2005297 ,
         0.13797034,  0.99999994]], dtype=float32)

In [29]:
dataset_to_modify=all_plots[:15]
with open('unbiased_dataset.txt','r+'):
    f=open('unbiased_dataset.txt','r+')
    for i in range(len(dataset_to_modify)):
        f.write(dataset_to_modify[i])
        f.write('\n\n')
    f.close()

In [21]:
def gender_bias_loss(model, batch):
    input_ids = batch['input_ids']
    word_embeddings = model.get_input_embeddings()(input_ids)
    she_embedding = word_embeddings[:, tokenizer.encode('she')[0], :]
    he_embedding = word_embeddings[:, tokenizer.encode('he')[0], :]
    avg_similarity = F.cosine_similarity(word_embeddings, she_embedding, dim=2).mean() - F.cosine_similarity(word_embeddings, he_embedding, dim=2).mean()
    return avg_similarity

In [24]:
def fine_tune(model, tokenizer, dataset, num_epochs, learning_rate):
    model.train()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch in dataset:
            optimizer.zero_grad()
            loss = gender_bias_loss(model, batch) + model.compute_loss(batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataset)}')

In [ ]:
# def equalize_gender_bias(text):
#     tokens = tokenizer.tokenize(text)
#     input_ids = tokenizer.convert_tokens_to_ids(tokens)
#     with torch.no_grad():
#         embeddings = model.embeddings.word_embeddings(torch.tensor(input_ids))
#         # Modify embeddings for gender-specific terms
#         # (e.g., replace "he" with "she" or vice versa)
#         # Adjust the embeddings as needed to reduce bias
#     return modified_text